In [115]:
#loading libraries 
import pandas as pd
import numpy as np
import time
import tweepy
import re
import json
import glob
from tqdm.notebook import tqdm
import string
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))

from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/evgeniya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/evgeniya/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [183]:
#read json files append them and save in one pd file

mask_tweets = []

for filepath in tqdm(glob.glob('../tweeter/*.json')):
    title = filepath.split('.json*')
    #print(filepath)
    #with open(filepath) as f:
        #if title not in mask_tweets:
    df = pd.read_json(filepath)
    mask_tweets.append(df)
df_tweets=pd.concat(mask_tweets)

  0%|          | 0/19 [00:00<?, ?it/s]

In [185]:
df_tweets.head()

,source,public_metrics,created_at,author_id,geo,text,lang,id,withheld
0,Twitter for iPhone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-05-24 02:59:54+00:00,869616089831026688,{'place_id': '11ec8106d4d1a9a4'},There are generally other masking requirements...,en,1528933780056158208,NaN
1,twittbot.net,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-05-24 02:59:53+00:00,515162271,NaN,"""if u wear a mask, u can naver be happy"" Mask",en,1528933778886111232,NaN
2,Twitter Web App,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-05-24 02:59:44+00:00,862789703875018752,NaN,@Glen79644490 @SandroDemaio Masking almost wip...,en,1528933740537253888,NaN
3,Twitter for Android,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-05-24 02:59:08+00:00,824710543428636672,NaN,#FFS\n\nMy county has the highest rate in our ...,en,1528933587440918528,NaN
4,Twitter Web App,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-05-24 02:58:50+00:00,3299521675,NaN,"@mehdirhasan Double-vaxxed, no Covid. But, wi...",en,1528933514820845568,NaN


In [186]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97468 entries, 0 to 8275
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   source          97468 non-null  object             
 1   public_metrics  97468 non-null  object             
 2   created_at      97468 non-null  datetime64[ns, UTC]
 3   author_id       97468 non-null  int64              
 4   geo             1548 non-null   object             
 5   text            97468 non-null  object             
 6   lang            97468 non-null  object             
 7   id              97468 non-null  int64              
 8   withheld        1 non-null      object             
dtypes: datetime64[ns, UTC](1), int64(2), object(6)
memory usage: 7.4+ MB


In [69]:
#Clean the text
str_ing = "@Glen79644490 @SandroDemaio Masking almost wiped out the flu over the past couple of years. This isn't anything unexpected. It was always going to come roaring back the first flu season masks were largely discarded."
re.sub(r'@[A-Za-z0-9]+', '', str_ing) #removing @mentions

"  Masking almost wiped out the flu over the past couple of years. This isn't anything unexpected. It was always going to come roaring back the first flu season masks were largely discarded."

In [27]:
re.sub(r'#', '', str_ing) #removing '#' symbol

'@CzarBulldawgRic He is SMART to wear a MASK to..'

In [ ]:
re.sub(r'RT[\s]+', '', str_ing)

In [30]:
#remove URL links (httpxxx)
test = 'http://www.kkk.com jskll;;'
re.sub(r'https?://[A-Za-z0-9./]*','', test)

' jskll;;'

In [62]:
txt=df_tweets['text']

In [203]:
def cleanTxt(txt):
    local_txt = re.sub(r'@[A-Za-z0-9]+', '', txt) #removing @mentions
    local_txt = re.sub(r'#', '', local_txt) #removing '#' symbol
    local_txt = re.sub(r'RT[\s]+', '', local_txt)
    local_txt = re.sub(r'https?://[A-Za-z0-9./]*','', local_txt)#removing the hyperlink
    local_txt = re.sub('[^A-Z a-z 0-9-]+', '', local_txt) #removing special characters
    local_txt = local_txt.translate(str.maketrans('', '', string.punctuation))
    tt = TweetTokenizer()
    tokens =tt.tokenize(local_txt)
    no_stops = [w for w in tokens if not w in stop_words]
    local_txt_cleaned = [w for w in no_stops if not w in ('amp', 'rt')]
    return " ".join(local_txt_cleaned)
    
  

In [71]:
cleanTxt(txt.values[2])

"  Masking almost wiped out the flu over the past couple of years. This isn't anything unexpected. It was always going to come roaring back the first flu season masks were largely discarded."

In [204]:
df_tweets['text'] = df_tweets['text'].apply(cleanTxt)

In [205]:
df_tweets['text'] = df_tweets['text'].str.lower()


In [206]:
df_tweets

,source,public_metrics,created_at,author_id,geo,text,lang,id,withheld
0,Twitter for iPhone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-05-24 02:59:54+00:00,869616089831026688,{'place_id': '11ec8106d4d1a9a4'},generally masking requirement cross border tra...,en,1528933780056158208,NaN
1,twittbot.net,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-05-24 02:59:53+00:00,515162271,NaN,u wear mask u naver happy mask,en,1528933778886111232,NaN
2,Twitter Web App,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-05-24 02:59:44+00:00,862789703875018752,NaN,masking almost wiped flu past couple year isnt...,en,1528933740537253888,NaN
3,Twitter for Android,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-05-24 02:59:08+00:00,824710543428636672,NaN,ffsmy county highest rate regionive never stop...,en,1528933587440918528,NaN
4,Twitter Web App,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-05-24 02:58:50+00:00,3299521675,NaN,doublevaxxed covid taking another vaccine stil...,en,1528933514820845568,NaN
...,...,...,...,...,...,...,...,...,...
8271,Twitter for iPhone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-06-02 03:00:45+00:00,230044693,NaN,easiest answer cancel truth seems isnt possibl...,en,1532195485011812352,NaN
8272,Twitter Web App,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-06-02 03:00:27+00:00,1044661675100532736,NaN,smart safe rule thumb use possibility error ju...,en,1532195411217108992,NaN
8273,TiwttCloud,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-06-02 03:00:22+00:00,840816291204202496,NaN,city reissued mask mandate everyonehealth coro...,en,1532195389687812096,NaN
8274,Twitter for Android,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-06-02 03:00:09+00:00,13701362,NaN,leech hhs science prof universal esp high filt...,en,1532195337653235712,NaN


In [214]:
#lemmatization/stemming
lemmatizer = WordNetLemmatizer()
def lemmatizing(tw):
    lemma_words = [lemmatizer.lemmatize(word) for word in tw]
    return "".join(lemma_words)

In [215]:
df_tweets['text'] = df_tweets['text'].apply(lambda x:lemmatizing(x))

In [216]:
df_tweets

,source,public_metrics,created_at,author_id,geo,text,lang,id,withheld
0,Twitter for iPhone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-05-24 02:59:54+00:00,869616089831026688,{'place_id': '11ec8106d4d1a9a4'},generally masking requirement cross border tra...,en,1528933780056158208,NaN
1,twittbot.net,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-05-24 02:59:53+00:00,515162271,NaN,u wear mask u naver happy mask,en,1528933778886111232,NaN
2,Twitter Web App,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-05-24 02:59:44+00:00,862789703875018752,NaN,masking almost wiped flu past couple year isnt...,en,1528933740537253888,NaN
3,Twitter for Android,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-05-24 02:59:08+00:00,824710543428636672,NaN,ffsmy county highest rate regionive never stop...,en,1528933587440918528,NaN
4,Twitter Web App,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-05-24 02:58:50+00:00,3299521675,NaN,doublevaxxed covid taking another vaccine stil...,en,1528933514820845568,NaN
...,...,...,...,...,...,...,...,...,...
8271,Twitter for iPhone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-06-02 03:00:45+00:00,230044693,NaN,easiest answer cancel truth seems isnt possibl...,en,1532195485011812352,NaN
8272,Twitter Web App,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-06-02 03:00:27+00:00,1044661675100532736,NaN,smart safe rule thumb use possibility error ju...,en,1532195411217108992,NaN
8273,TiwttCloud,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-06-02 03:00:22+00:00,840816291204202496,NaN,city reissued mask mandate everyonehealth coro...,en,1532195389687812096,NaN
8274,Twitter for Android,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-06-02 03:00:09+00:00,13701362,NaN,leech hhs science prof universal esp high filt...,en,1532195337653235712,NaN


In [217]:
df_tweets_cleaned = df_tweets[['text']]

In [218]:
df_tweets_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97468 entries, 0 to 8275
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    97468 non-null  object
dtypes: object(1)
memory usage: 1.5+ MB


In [219]:
df_tweets_cleaned.drop_duplicates(inplace =True)

/home/evgeniya/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [220]:
df_tweets_cleaned.to_csv('../tweeter/df_tweets.csv')

In [133]:
#df_tweets.to_csv('../tweeter/df_tweets.csv')